# Dynamic Ensembling for Knowledge Graph Completion

This notebook can be used to obtain results for dynamic ensembling of SimKGC and RotatE on the WN18RR and CoDex-M datasets.

## Import PyG and other required libraries

In [ ]:
import os.path as osp
import torch
import torch.optim as optim
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric
!pip install ogb
!pip install faiss-gpu
from torch_geometric.datasets import WordNet18RR
from torch_geometric.nn import RotatE
from tqdm import tqdm
import pickle as pkl
from torch_geometric.data import Data
import torch.nn as nn
from torch_geometric.utils import index_sort
import json
from itertools import chain
import pickle

## Link the Colab to your Google Drive

We use Google Drive to load datasets and embeddings.

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

## Load all Global Variables

For this colab, we need to load trained SimKGC embeddings, entity and relation mappings for SimKGC, as well as the trained RotatE model.

1. Please change DATA_ROOT to the path to the SimKGC version of your dataset. For WN18RR, this is available at https://drive.google.com/drive/folders/1tnYam16BA1IHYDCQgzqcoRLuBOkizSu2?usp=sharing. For CoDex, this is available at https://drive.google.com/drive/folders/1MUnSq7ENTIV7nah3IiCqfV8Wb3McAgWc?usp=drive_link.
2. Please change ROTATE_ROOT to the path to your trained RotatE model.
3. Please change SIMKGC_DUMP to the path to the SimKGC embeddings. For WN18RR, this is available at https://drive.google.com/drive/folders/1R0c5W0ofiznQQ8mjsJj0SUohFWrpOc_C?usp=sharing. For CoDex, this is available at https://drive.google.com/drive/folders/1PfXRgT8xzxXXhXoc81M86f1D9q67JFNc?usp=sharing.

In [ ]:
DATASET = 'WN18RR' # Either CodexM or WN18RR
DATA_ROOT = "/content/drive/Shareddrives/CS224W Project/WN18RR/" # Path to the SimKGC version of the dataset
ROTATE_ROOT = "/content/drive/Shareddrives/CS224W Project/kgc_models/wn18rr.mdl" # Path to the dumped RotatE model
SIMKGC_DUMP = "/content/drive/Shareddrives/CS224W Project/WN18RR_Vectors" # Path to the dumped SimKGC embeddings
CHANNEL = 500 # Hidden dimensionality for RotatE
MARGIN = 9.0 # Margin for RotatE score computation
EPOCHS = 2 # Number of epochs of training for the dynamic ensemble
LR = 5.0e-4 # Learning rate for training the dynamic ensemble
BATCH_SIZE = 64 # Batch size for training the dynamic ensemble
NUM_NEGATIVES = 2000 # Number of negative samples per training example

## Dataset Loading

In [ ]:
# The WN18RR relation mappings for PyG

WN18RR_E2ID = {
        '_also_see': 0,
        '_derivationally_related_form': 1,
        '_has_part': 2,
        '_hypernym': 3,
        '_instance_hypernym': 4,
        '_member_meronym': 5,
        '_member_of_domain_region': 6,
        '_member_of_domain_usage': 7,
        '_similar_to': 8,
        '_synset_domain_topic_of': 9,
        '_verb_group': 10,
    }

In [ ]:
'''
For this setting, we load both CoDex-M and WN18RR manually, without relying on PyG.
This is because we need the entity id and relation id mappings to align the SimKGC embeddings to the RotatE embeddings
'''

device = f'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

node2id, rel2id, idx, relidx = {}, {}, 0, 0

srcs, dsts, edge_types = [], [], []
for file in ["train.txt", "valid.txt", "test.txt"]:
    with open(DATA_ROOT + file, 'r') as f:
        data = f.read().split()

        src = data[::3]
        dst = data[2::3]
        edge_type = data[1::3]

        for i in chain(src, dst):
            if i not in node2id:
                node2id[i] = idx
                idx += 1

        if (DATASET == 'CodexM'):
            for i in edge_type:
                if i not in rel2id:
                    rel2id[i] = relidx
                    relidx += 1
        else:
            rel2id = WN18RR_E2ID

        src = [node2id[i] for i in src]
        dst = [node2id[i] for i in dst]
        edge_type = [rel2id[i] for i in edge_type]

        srcs.append(torch.tensor(src, dtype=torch.long))
        dsts.append(torch.tensor(dst, dtype=torch.long))
        edge_types.append(torch.tensor(edge_type, dtype=torch.long))

src = torch.cat(srcs, dim=0)
dst = torch.cat(dsts, dim=0)
edge_type = torch.cat(edge_types, dim=0)

train_mask = torch.zeros(src.size(0), dtype=torch.bool)
train_mask[:srcs[0].size(0)] = True
val_mask = torch.zeros(src.size(0), dtype=torch.bool)
val_mask[srcs[0].size(0):srcs[0].size(0) + srcs[1].size(0)] = True
test_mask = torch.zeros(src.size(0), dtype=torch.bool)
test_mask[srcs[0].size(0) + srcs[1].size(0):] = True

num_nodes = max(int(src.max()), int(dst.max())) + 1
_, perm = index_sort(num_nodes * src + dst)

edge_index = torch.stack([src[perm], dst[perm]], dim=0)
edge_type = edge_type[perm]
train_mask = train_mask[perm]
val_mask = val_mask[perm]
test_mask = test_mask[perm]

data = Data(edge_index=edge_index, edge_type=edge_type,
            train_mask=train_mask, val_mask=val_mask,
            test_mask=test_mask, num_nodes=num_nodes)

data

In [ ]:
# To compute the evaluation metrics in the filtered setting, we need to compute neighbours for all nodes in the knowledge graph.

neighbours = [[set() for _ in range(2*data.num_edge_types)] for _ in range(data.num_nodes)]

for idx in tqdm(range(len(data.edge_type))):
    '''
    We will add a (t,r^{-1},h) edge for all (h,r,t) in the graph later. The relation ID for r^{-1} is data.num_edge_types + the relation ID for r.
    Therefore, if t is connected to h through the relation r, h will be connected to t through the relation r^{-1}.
    '''
    neighbours[data.edge_index[0, idx].item()][data.edge_type[idx].item()].add(data.edge_index[1, idx].item())
    neighbours[data.edge_index[1, idx].item()][data.num_edge_types + data.edge_type[idx].item()].add(data.edge_index[0, idx].item())

## Load the RotatE model from disk

In [ ]:
'''
We will model (?,r,t) queries as (t,r^{-1},?) queries.
Since we have the inverse relation r^{-1} for each edge r, we double the number of relations below.
'''

model = RotatE(
            num_nodes=data.num_nodes,
            num_relations=2*data.num_edge_types,
            hidden_channels=CHANNEL,
            margin=MARGIN
        ).to(device)

model.load_state_dict(torch.load(ROTATE_ROOT))
model.eval()

## Create mapping of SimKGC and RotatE entity and relation IDs

In [ ]:
'''
SimKGC and RotatE are trained on different versions of the datasets.
Therefore, we create a mapping between entity and relation IDs from these different versions here
'''

ent = json.load(open(DATA_ROOT + 'entities.json', 'r'))
siment = [_["entity_id"] for _ in ent]
siment = {siment[_]:_ for _ in range(len(siment))}
nbf2rnnent = {}
for _ in node2id:
    nbf2rnnent[node2id[_]] = siment[_]

rel = json.load(open(DATA_ROOT + 'relations.json', 'r'))
simrel = list(rel.keys())
simrel = {simrel[_]:_ for _ in range(len(simrel))}
nbf2rnnrel = {}
dump_idxs = len(rel2id)
for _ in simrel:
    if _ in rel2id:
        nbf2rnnrel[rel2id[_]] = simrel[_]
        nbf2rnnrel[len(simrel) + rel2id[_]] = len(simrel) + simrel[_]
    else:
        nbf2rnnrel[dump_idxs] = simrel[_]
        nbf2rnnrel[len(simrel) + dump_idxs] = len(simrel) + simrel[_]
        dump_idxs += 1

## Load SimKGC embeddings

In [ ]:
# Instantiate SimKGC class and load embeddings

class SimKGC():
    """
    A class for loading SimKGC embeddings and computing SimKGC scores

    """
    def __init__(self, nbf2rnnent, nbf2rnnrel):
        # Initialize the class using the id mappings created above

        self.h_embs = []
        self.re_index = [nbf2rnnent[_] for _ in range(len(nbf2rnnent))]
        self.re_rel = [nbf2rnnrel[_] for _ in range(len(nbf2rnnrel))]

        # Load tail embeddings
        t_file = f'{SIMKGC_DUMP}/SimKGC_t_rep.pkl'
        t_emb = pickle.load(open(t_file, 'rb'))
        t_emb = t_emb[self.re_index, :]
        self.t_emb = t_emb.t().cpu()
        print(self.t_emb.shape)
        print(f'Loaded embeddings for tail')

        # Load (head,relation) embeddings
        for _ in self.re_rel:
            print(f'Loading for relation {_}')
            h_file = f'{SIMKGC_DUMP}/SimKGC_h_{_}_rep.pkl'
            h_emb = pickle.load(open(h_file, 'rb'))
            h_emb = h_emb[self.re_index, :]
            self.h_embs.append(h_emb.cpu())
            print(f'Loaded embeddings for relation {_}')
        self.h_embs = torch.stack(self.h_embs).cpu()
        print(self.h_embs.shape)

    def get_h(self, h, r):
        """
        Computes the SimKGC score from a vector of head and relation IDs for all entities in the dataset as tails

        Parameters:
        -----------
        h : torch.Tensor
            Head IDs
        r : torch.Tensor
            Relation IDs

        Returns:
        --------
        torch.Tensor
            Batched score distribution for each query.
        """
        head = h
        rel = r
        h_req = self.h_embs[rel, head]
        t_req = self.t_emb
        result = torch.matmul(h_req, t_req)
        return result

simkgc = SimKGC(nbf2rnnent, nbf2rnnrel)

## Create Class for Dynamic Ensemble Computation

In [ ]:
import json
import numpy as np

class Selector(nn.Module):
    """
    A class for learning a dynamic ensemble of base KGC models.

    """
    def __init__(self, simkgc, rotate, num_layers, hidden_dims, input_dim):
        # Initialize the class with the base models and the hyperparameters for the MLP
        super(Selector, self).__init__()

        self.simkgc = simkgc
        self.rotate = rotate
        self.rotate.requires_grad_(False)

        mlp = []
        # Input is a tensor containing the distribution of scores from the base model over all candidate tail entities
        mlp.append(nn.Linear(input_dim, hidden_dims[0]))
        for i in range(num_layers - 1):
            layer = nn.Linear(hidden_dims[i], hidden_dims[i+1])
            mlp.append(layer)
            mlp.append(nn.ReLU())
        # Output is the ensemble weight
        mlp.append(nn.Linear(hidden_dims[-1], 1))
        self.mlp = nn.Sequential(*mlp)

    def get_features_and_normalize(self, score):
        # Max-min normalize the score distribution
        pre_mins = torch.amin(score, dim=1).unsqueeze(dim = -1)
        score = (score - pre_mins)
        maxs = torch.amax(score, dim=1).unsqueeze(dim = -1)
        score = score/maxs
        # Compute the mean of the distribution
        means = torch.mean(score, dim=1).unsqueeze(dim = -1)
        # Compute the mean of the top 10 scores in the distribution
        topk = torch.mean(torch.topk(score, dim=1, k=10, largest=True).values, dim=1).unsqueeze(dim = -1)
        return score, [mdiffs, topk]

    def get_rotate_scores(self, h_index, r_index):
        # Run the PyG RotatE model on the given (h,r) pairs to get the score distribution
        arange = range(h_index.numel())
        tot_scores = []

        for i in arange:
            h, r = h_index[i], r_index[i]
            scores = []
            tail_indices = torch.arange(len(nbf2rnnent), device=h.device)
            for ts in tail_indices.split(20_000):
                scores.append(self.rotate(h.expand_as(ts), r.expand_as(ts), ts))
            scores = torch.cat(scores)
            tot_scores.append(scores.unsqueeze(0))

        tot_scores = torch.cat(tot_scores, dim=0)
        return tot_scores

    def forward(self, h_index, r_index):
        # Accumulate the distribution features from all base models
        features = []

        # Get the SimKGC scores and features
        simkgc_score = self.simkgc.get_h(h_index.cpu(), r_index.cpu()).to(h_index.device)
        simkgc_score, feature = self.get_features_and_normalize(simkgc_score)
        features += feature

        # Get the RotatE scores and features
        rotate_score = self.get_rotate_scores(h_index, r_index)
        rotate_score, feature = self.get_features_and_normalize(rotate_score)
        features += feature

        # Concatenate features to get input tensor for MLP
        mlp_in = torch.cat(tuple(features), dim = 1)
        weight = self.mlp(mlp_in.to(h_index.device))

        # Return the score distribution after dynamic ensembling
        return simkgc_score + weight*rotate_score

In [ ]:
# Instantiate the Selector class with the default hyperparameters

selector = Selector(simkgc, model, 2, [32,32], 4)

## Create Dataloader

In [ ]:
# Create Dataloader

# (t,r^{-1},h) edges are added for all (h,r,t) in the graph

loader = model.loader(
            head_index=torch.cat([data.edge_index[0, data.val_mask],data.edge_index[1, data.val_mask]], dim=0),
            rel_type=torch.cat([data.edge_type[data.val_mask], data.num_edge_types+data.edge_type[data.val_mask]], dim=0),
            tail_index=torch.cat([data.edge_index[1, data.val_mask], data.edge_index[0, data.val_mask]], dim=0),
            batch_size=BATCH_SIZE,
            shuffle=True
        )

## Function to run the training loop

In [ ]:
def train(selector, loader, optimizer):
    # Run the training loop for one epoch
    selector.train()

    for head_index, rel_type, tail_index in tqdm(loader):
        optimizer.zero_grad()
        pred = selector(head_index.to(device), rel_type.to(device))

        req_scores = pred.gather(1, tail_index.unsqueeze(0).to(device)).squeeze().unsqueeze(1)
        # Negative sampling
        sampled_negatives = torch.randperm(pred.size(dim=-1))[:NUM_NEGATIVES].to(device)
        sampled_scores = pred[:, sampled_negatives]

        # Compute loss and backpropagate
        loss = nn.CrossEntropyLoss()
        loss_input = torch.cat([req_scores, sampled_scores], dim = 1)
        loss_target = torch.zeros(pred.size(dim=0)).long()
        final_loss = loss(loss_input, loss_target.to(device))
        final_loss.backward()
        optimizer.step()

## Function to run testing

In [ ]:
@torch.no_grad()
def test(selector, data):
    '''
    Testing in the filtered setting according to (Bordes et al, 2013)[https://proceedings.neurips.cc/paper/2013/file/1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf
    '''

    selector.eval()
    # Augment the testing data with inverse relations
    head_index=torch.cat([data.edge_index[0, data.test_mask],data.edge_index[1, data.test_mask]], dim=0).to(device)
    rel_type=torch.cat([data.edge_type[data.test_mask], data.num_edge_types+data.edge_type[data.test_mask]], dim=0).to(device)
    tail_index=torch.cat([data.edge_index[1, data.test_mask], data.edge_index[0, data.test_mask]], dim=0).to(device)
    arange = range(head_index.numel())
    arange = tqdm(arange)

    mean_ranks, reciprocal_ranks, hits_at_1, hits_at_10 = [], [], [], []
    for i in arange:
        h, r, t = head_index[i], rel_type[i], tail_index[i]
        # Get dynamic ensembling scores for all entities in the dataset as tails
        flattened_scores = selector(torch.tensor([h]).to(device),torch.tensor([r]).to(device)).squeeze()

        # Filter out neighbours from candidate tails
        curr_neighbours = list(neighbours[h.item()][r.item()])
        mask_indices = []
        for e_id in curr_neighbours:
            if e_id == t.item():
                continue
            mask_indices.append(e_id)
        mask_indices = torch.LongTensor(mask_indices).to(device)
        flattened_scores.index_fill_(0, mask_indices, -1).cpu()

        # Compute rank, mrr and hits@k
        rank = int((flattened_scores.argsort(
                descending=True) == t).nonzero().view(-1))
        mean_ranks.append(rank)
        reciprocal_ranks.append(1 / (rank + 1))
        hits_at_1.append(rank < 1)
        hits_at_10.append(rank < 10)

    # Accumulate results from all queries
    mean_rank = float(torch.tensor(mean_ranks, dtype=torch.float).mean())
    mrr = float(torch.tensor(reciprocal_ranks, dtype=torch.float).mean())
    hits_at_1 = int(torch.tensor(hits_at_1).sum()) / len(hits_at_1)
    hits_at_10 = int(torch.tensor(hits_at_10).sum()) / len(hits_at_10)
    print(mean_rank, mrr, hits_at_1, hits_at_10)

## Run the training loop!

In [ ]:
# Initialize the Optimizer

selector.to(device)
optimizer = optim.Adam(selector.parameters(), lr=LR)

In [ ]:
# Training loop

for ep in range(EPOCHS):
    train(selector, loader, optimizer)
    test(selector, data)